pip install qrisp 

https://www.qrisp.eu

QPE implementation in Qrisp: https://qrisp.eu/reference/Primitives/QPE.html

Experiments performed on QITE_experiments branch: https://github.com/eclipse-qrisp/Qrisp/tree/QITE_experiments with lowered simulator cutoff_ratio for high precision state vector simulations


In [ ]:
from qrisp import QuantumVariable, QPE
from qrisp.operators import X, Y, Z
from qrisp.vqe import VQEProblem
from qrisp.vqe.problems.heisenberg import create_heisenberg_init_function

import networkx as nx
import numpy as np
from scipy.sparse.linalg import eigsh, svds

import pickle
from datetime import datetime

def save_data(data, filename):  
    # Open a file for writing
    with open(filename+'.pickle', 'wb') as file:
        # Write the object to the file
        pickle.dump(data, file)

def load_data(filename):
    with open(filename+'.pickle', 'rb') as file:
        # Load the object from the file
        data = pickle.load(file)
    return data

# Heisenberg XXX model

$$ H_{\text{TFIM}} = \sum_{j=0}^{L-1}(X_jX_{j+1}+Y_jY_{j+1}+Z_jZ_{j+1}) $$


In [ ]:
def rescaled_Hamiltonian(G, scaling_factor=1):

    H = sum(X(i)*X(j)+Y(i)*Y(j)+Z(i)*Z(j) for (i,j) in G.edges())

    M = H.to_sparse_matrix()
    eigenvalues, eigenvectors = eigsh(M, k=1, which='SA')
    E0 = eigenvalues[0]

    # Rescale to E0=0
    H2 = H - E0

    M = H2.to_sparse_matrix()
    u, s, vt = svds(M, k=1, which='LM')
    spectral_norm = s[0]

    E_max = spectral_norm

    # Rescale such that spectrum in [0,1)
    H3 = H2*(1/(E_max+1))*scaling_factor

    M = H3.to_sparse_matrix()
    eigenvalues, eigenvectors = eigsh(M, k=2, which='SA')

    # Ground state
    psi_0 = eigenvectors[:,0]

    E0 = eigenvalues[0]
    E1 = eigenvalues[1]

    delta = E1-E0
    print('Spectral gap:', delta)
    print('Ground state energy:', E0)

    return H3, psi_0


def create_U0(G, init_type):

    H = sum(X(i)*X(j)+Y(i)*Y(j)+Z(i)*Z(j) for (i,j) in G.edges())

    if init_type=='Singlet':
        # Tensor product of singlet states
        M = nx.maximal_matching(G)
        U_singlet = create_heisenberg_init_function(M)
        return U_singlet

    if init_type=='HVA':
        # HVA warm start
        M = nx.maximal_matching(G)
        U_singlet = create_heisenberg_init_function(M)

        H0 = sum((X(i)*X(j)+Y(i)*Y(j)+Z(i)*Z(j)) for i,j in M)
        H1 = sum((X(i)*X(j)+Y(i)*Y(j)+Z(i)*Z(j)) for i,j in set(G.edges())-M)

        def ansatz(qv,theta):
            H1.trotterization(method='commuting')(qv, t=theta[1]/4)
            H0.trotterization(method='commuting')(qv, t=theta[0]/4)

        HVA = VQEProblem(H, ansatz, 2, init_function=U_singlet)
        U_HVA = HVA.train_function(QuantumVariable(G.number_of_nodes()), depth=1, max_iter=100)
        return U_HVA
    

def create_instance(L, init_type='Singlet', scaling_factor=1):

    # Create a graph
    G = nx.Graph()
    G.add_edges_from([(k,k+1) for k in range(L-1)]) 

    H_rescaled, psi_0 = rescaled_Hamiltonian(G, scaling_factor)

    U_0 = create_U0(G, init_type)

    qv = QuantumVariable(G.number_of_nodes())
    U_0(qv)

    psi = qv.qs.statevector_array()
    F0 = np.abs(np.dot(psi.conj().transpose(),psi_0))**2
    print('Initial Fidelity', F0)

    return H_rescaled, U_0, psi_0

## Ground state preparation with QPE for known ground state energy

The spectrum of the Hamiltonian is rescaled to $[0,1)$ with $\lambda_0=0$. Preparing the ground state corresponds to measuring 0 in precision qubits for QPE.

In [ ]:
def prepare_ground_state(precision, H, U_0, psi_0):

    result = dict()

    qv = QuantumVariable(H.find_minimal_qubit_amount())
    U_0(qv)

    # Hamiltonian simulation via second order Suzuki-Trotter formula with 2 steps 
    exp_H = H.trotterization(order=2, forward_evolution=False)

    qpe_res = QPE(qv, exp_H, precision=precision, kwargs={"t":2*np.pi, "steps":2}, iter_spec=True)

    qc = qpe_res.qs.compile()
    tqc = qc.transpile(basis_gates=["cz","u"])

    result["ops"] = tqc.count_ops()
    result["depth"] = tqc.depth()

    results = qpe_res.get_measurement(precompiled_qc=qc)
    sorted_results = dict(sorted(results.items(), key=lambda item: item[1], reverse=True))
    #print(sorted_results)
    P0 = sorted_results[0]
    #print('Success probability:', P0)
    result["P0"] = P0

    #qc = qpe_res.qs.compile()

    n = H.find_minimal_qubit_amount()

    for i in range(qc.num_qubits() - n):
        qc.qubits.insert(0, qc.qubits.pop(-1))

    phi = qc.statevector_array()[:2**n]
    phi = phi/np.linalg.norm(phi)

    F0 = (np.abs(np.dot(phi.conj().transpose(),psi_0))**2)
    #print('Fidelity:', F0)
    result["F0"] = F0

    return result


In [ ]:
def benchmark(L, init_type='Singlet', scaling_factor=1):

    H_rescaled, U_0, psi_0 = create_instance(L, init_type, scaling_factor)
    results_dict = dict()
    precision_range = [1,2,3,4,5]
    for precision in precision_range:
        results_dict[precision] = prepare_ground_state(precision, H_rescaled, U_0, psi_0)

    return results_dict

benchmark_results = dict()
for L in range(10,20,2):
    res = benchmark(L, init_type='Singlet', scaling_factor=1)
    benchmark_results[L] = res


In [ ]:
# Save benchmark data
current_datetime = datetime.now()
date_time_string = current_datetime.strftime("%m-%d-%H")
#save_data(benchmark_results,'data/benchmarks_QPE_Singlet_'+date_time_string)
#save_data(benchmark_results,'data/benchmarks_QPE_HVA_'+date_time_string)